<p align="center"><h1 align="center">Adcanced Machine Learning Models Applied to COVID Diagnosing <br> Additional Model: Inceptionv3 Model<br>Spring 2024 <br> Nicholas Choong, Qiankun Li</h1>

---
<br>

### In this notebook we train the Inceptionv3_model proposed by [1]

# **STEP 1: Import packages and extract the dataset**



In [1]:
!pip install opencv-python-headless

In [2]:
# Load libraries and then download data
import pandas as pd
import sys
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import zipfile
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from itertools import repeat
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3

In [3]:
# Specify the path to your dataset zip file
zip_file_path = r'C:\Users\l\Downloads\COVID-19_Radiography_Dataset.zip'

# Specify the extraction path, the empty file location you want to put your files in 
extraction_path = r'C:\Users\l\Downloads\Extracted Dataset'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Dataset extracted successfully.")

Dataset extracted successfully.


# **STEP 2: Prepare the dataset**

In [4]:
# Extracting all filenames iteratively
base_path = r'C:\Users\l\Downloads\Extracted Dataset\COVID-19_Radiography_Dataset'
categories = ['COVID/images', 'Normal/images', 'Viral Pneumonia/images']

# load file names to fnames list object
fnames = []
for category in categories:
    image_folder = os.path.join(base_path, category)
    file_names = os.listdir(image_folder)
    full_path = [os.path.join(image_folder, file_name) for file_name in file_names]
    fnames.append(full_path)


print('number of images for each category:', [len(f) for f in fnames])
#print(fnames[0:2]) #examples of file names

number of images for each category: [3616, 10192, 1345]


In [5]:
#Reduce number of images to first 1345 for each category
fnames[0] = fnames[0][0:1344]
fnames[1] = fnames[1][0:1344]
fnames[2] = fnames[2][0:1344]

# Reduce number of images to first 1345 for each category to balance the dataset
fnames = [f[:1344] for f in fnames]

In [6]:
# Import image, load to array of shape height, width, channels, then min/max transform.
# Write preprocessor that will match up with model's expected input shape.
from keras.preprocessing import image
import numpy as np
from PIL import Image

# Define the image preprocessor
def preprocessor(img_path):
    img = Image.open(img_path).convert("RGB").resize((192, 192))  # Import image, ensure it's in RGB and resize.
    img = (np.float32(img) - 1.) / (255 - 1.)  # Min-max transform.
    img = img.reshape((192, 192, 3))  # Create final shape as array with correct dimensions for Keras.
    return img

# Try on a single image file (imports file and preprocesses it to data with the expected shape)
image_path = os.path.join(base_path, 'COVID/images/COVID-2273.png')
img_shape = preprocessor(image_path).shape
print('Processed image shape:', img_shape)

Processed image shape: (192, 192, 3)


In [7]:
#Import image files iteratively and preprocess them into array of correctly structured data
# Create list of file paths
image_filepaths=fnames[0]+fnames[1]+fnames[2]

# Iteratively import and preprocess data using map function
# map functions apply your preprocessor function one step at a time to each filepath
preprocessed_image_data=list(map(preprocessor, image_filepaths))

# Object needs to be an array rather than a list for Keras (map returns to list object)
X = np.array(preprocessed_image_data) # Assigning to X to highlight that this represents feature input data for our model

In [8]:
# Create y data made up of correctly ordered labels from file folders
from itertools import repeat

# Recall that we have five folders with the following number of images in each folder
#...corresponding to each flower type
print('number of images for each category:', [len(f) for f in fnames])
covid=list(repeat("COVID", 1344))
normal=list(repeat("NORMAL", 1344))
pneumonia=list(repeat("PNEUMONIA", 1344))

#combine into single list of y labels
y_labels = covid + normal + pneumonia

#check length, same as X above
print(len(y_labels))

# Need to one hot encode for Keras.  Let's use Pandas
y = pd.get_dummies(y_labels)

display(y)

number of images for each category: [1344, 1344, 1344]
4032


,COVID,NORMAL,PNEUMONIA
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
4027,False,False,True
4028,False,False,True
4029,False,False,True
4030,False,False,True


In [9]:
# Base_path is the path your data is extracted to
base_path = r'C:\Users\l\Downloads\Extracted Dataset\COVID-19_Radiography_Dataset'
categories = ['COVID/images', 'Normal/images', 'Viral Pneumonia/images']


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setting up the data augmentation configuration
data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)


# **STEP 3: Splitting Dataset**

In [11]:
# ======Train test split resized images (Hackathon Note!! Use same train test split to be able to submit predictions to leaderboard!)=======================================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.32, random_state = 1987)
X_test.shape, y_test.shape

((1291, 192, 192, 3), (1291, 3))

In [12]:
#Clear objects from memory
del(X)
del(y)
del(preprocessed_image_data)

In [13]:
#Save data to be able to reload quickly if memory crashes or if you run Runtime>Restart Runtime
import pickle


# Open a file and use dump()
with open('X_train.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(X_train, file)


# Open a file and use dump()
with open('X_test.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(X_test, file)


# Open a file and use dump()
with open('y_train.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(y_train, file)


# Open a file and use dump()
with open('y_test.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(y_test, file)

In [14]:
#If you run out of Colab memory restart runtime, reload data and try again
import pickle

# Open the file in binary mode
with open('X_train.pkl', 'rb') as file:
    # Call load method to deserialze
    X_train = pickle.load(file)

# Open the file in binary mode
with open('y_train.pkl', 'rb') as file:
    # Call load method to deserialze
    y_train = pickle.load(file)

# **STEP 4: Model Training**

# Deploying inceptionv3_model

In [15]:
# Load the InceptionV3 model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(192, 192, 3))

# Freeze the base model layers
base_model.trainable = False

# Add a global average pooling layer
x = GlobalAveragePooling2D()(base_model.output)

# Add a dense output layer with 3 units for classification
outputs = Dense(3, activation='softmax')(x)

# Create the model
inceptionv3_model = Model(inputs=base_model.input, outputs=outputs)

# Print the model summary
inceptionv3_model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 192, 192, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 95, 95, 32)        │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 95, 95, 32)        │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 95, 95, 32)        │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 93, 93, 32)        │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 93, 93, 32)        │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 93, 93, 32)        │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 93, 93, 64)        │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 93, 93, 64)        │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 93, 93, 64)        │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 46, 46, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 46, 46, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 46, 46, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 46, 46, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 44, 44, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 21,808,931 (83.19 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
inceptionv3_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

callbacks_list_inceptionv3_model = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ModelCheckpoint(filepath='inceptionv3_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model
inceptionv3_model_train_history = inceptionv3_model.fit(
    X_train, y_train,
    epochs=25, #see the pic
    batch_size=32, #see the pic
    verbose=1,
    validation_split=0.2,
    callbacks=callbacks_list_inceptionv3_model
)

# Evaluate the model on the test set
test_loss, test_accuracy, test_precision, test_recall = inceptionv3_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test Precision: {test_precision}, Test Recall: {test_recall}")

Epoch 1/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.4108 - loss: 1.3814 - precision: 0.4399 - recall: 0.3179
Epoch 1: val_loss improved from inf to 0.81067, saving model to inceptionv3_model.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 397ms/step - accuracy: 0.4118 - loss: 1.3773 - precision: 0.4413 - recall: 0.3186 - val_accuracy: 0.6284 - val_loss: 0.8107 - val_precision: 0.6565 - val_recall: 0.5082 - learning_rate: 1.0000e-04
Epoch 2/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - accuracy: 0.6686 - loss: 0.7352 - precision: 0.7038 - recall: 0.5694
Epoch 2: val_loss improved from 0.81067 to 0.63721, saving model to inceptionv3_model.keras
69/69 ━━━━━━━━━━━━━━━━━━━━ 25s 356ms/step - accuracy: 0.6689 - loss: 0.7346 - precision: 0.7041 - recall: 0.5699 - val_accuracy: 0.7104 - val_loss: 0.6372 - val_precision: 0.7515 - val_recall: 0.6721 - learning_rate: 1.0000e-04
Epoch 3/25
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.7634 - loss: 0.5751 - precision: 0.7843 - recall: 0.

In [ ]:
# Visualize the training history for accuracy, precision, and recall
plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.plot(inceptionv3_model_train_history.history['accuracy'], label='Training Accuracy')
plt.plot(inceptionv3_model_train_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(inceptionv3_model_train_history.history['precision'], label='Training Precision')
plt.plot(inceptionv3_model_train_history.history['val_precision'], label='Validation Precision')
plt.title('Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(inceptionv3_model_train_history.history['recall'], label='Training Recall')
plt.plot(inceptionv3_model_train_history.history['val_recall'], label='Validation Recall')
plt.title('Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

plt.tight_layout()
plt.show()

# Reference

[1] "Rethinking the Inception Architecture for Computer Vision" - Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (CVPR 2016)
